要用 JScript 建置一個使用 ADODB.Connection 的物件導向 function 庫，並且能夠進行新增、修改、刪除、查詢操作，同時支持批次操作，以下是一個基本的實現範例。

假設你的環境支持 ActiveX 物件，以下是具體步驟：

設置全域變數 ConnectionString
創建 Database 類別
創建 executeQuery、executeNonQuery 和批次操作方法
使用 Watch.Log 進行日誌顯示
首先，設定全域變數 ConnectionString：

In [ ]:
var ConnectionString = "Your Connection String Here";


接下來，創建 Database 類別：

In [ ]:
function Database() {
    try {
        this.connection = new ActiveXObject("ADODB.Connection");
        this.connection.ConnectionString = ConnectionString;
        this.connection.Open();
        this.log("Database connection opened.", 3);
    } catch (e) {
        this.log("Error opening database connection: " + e.message, 1);
    }
}

Database.prototype.close = function() {
    try {
        this.connection.Close();
        this.log("Database connection closed.", 3);
    } catch (e) {
        this.log("Error closing database connection: " + e.message, 1);
    }
};

Database.prototype.log = function(message, level) {
    Watch.Log(message, level);
};


創建新增、修改、刪除、查詢的方法：

In [ ]:
Database.prototype.executeQuery = function(query) {
    var rs = new ActiveXObject("ADODB.Recordset");
    rs.Open(query, this.connection);
    var results = [];
    while (!rs.EOF) {
        var result = {};
        for (var i = 0; i < rs.Fields.Count; i++) {
            result[rs.Fields(i).Name] = rs.Fields(i).Value;
        }
        results.push(result);
        rs.MoveNext();
    }
    rs.Close();
    this.log("Query executed: " + query, 3);
    return results;
};

Database.prototype.executeNonQuery = function(query) {
    var cmd = new ActiveXObject("ADODB.Command");
    cmd.ActiveConnection = this.connection;
    cmd.CommandText = query;
    cmd.Execute();
    this.log("NonQuery executed: " + query, 3);
};

Database.prototype.insert = function(table, data) {
    var keys = [], values = [];
    for (var key in data) {
        keys.push(key);
        values.push("'" + data[key] + "'");
    }
    var query = "INSERT INTO " + table + " (" + keys.join(", ") + ") VALUES (" + values.join(", ") + ")";
    this.executeNonQuery(query);
};

Database.prototype.update = function(table, data, where) {
    var set = [];
    for (var key in data) {
        set.push(key + "='" + data[key] + "'");
    }
    var query = "UPDATE " + table + " SET " + set.join(", ") + " WHERE " + where;
    this.executeNonQuery(query);
};

Database.prototype.delete = function(table, where) {
    var query = "DELETE FROM " + table + " WHERE " + where;
    this.executeNonQuery(query);
};


創建批次操作的方法：

In [ ]:
Database.prototype.batchInsert = function(table, dataArray) {
    for (var i = 0; i < dataArray.length; i++) {
        this.insert(table, dataArray[i]);
    }
};

Database.prototype.batchUpdate = function(table, dataArray, whereArray) {
    for (var i = 0; i < dataArray.length; i++) {
        this.update(table, dataArray[i], whereArray[i]);
    }
};

Database.prototype.batchDelete = function(table, whereArray) {
    for (var i = 0; i < whereArray.length; i++) {
        this.delete(table, whereArray[i]);
    }
};


In [ ]:

Database.prototype.executeQuery = function(query) {
    try {
        var rs = new ActiveXObject("ADODB.Recordset");
        rs.Open(query, this.connection);
        var results = [];
        while (!rs.EOF) {
            var result = {};
            for (var i = 0; i < rs.Fields.Count; i++) {
                result[rs.Fields(i).Name] = rs.Fields(i).Value;
            }
            results.push(result);
            rs.MoveNext();
        }
        rs.Close();
        this.log("Query executed: " + query, 3);
        return results;
    } catch (e) {
        this.log("Error executing query: " + e.message + " Query: " + query, 1);
        return [];
    }
};

Database.prototype.executeNonQuery = function(query) {
    try {
        var cmd = new ActiveXObject("ADODB.Command");
        cmd.ActiveConnection = this.connection;
        cmd.CommandText = query;
        cmd.Execute();
        this.log("NonQuery executed: " + query, 3);
    } catch (e) {
        this.log("Error executing nonQuery: " + e.message + " Query: " + query, 1);
    }
};

Database.prototype.insert = function(table, data) {
    try {
        var keys = [], values = [];
        for (var key in data) {
            keys.push(key);
            values.push("'" + data[key] + "'");
        }
        var query = "INSERT INTO " + table + " (" + keys.join(", ") + ") VALUES (" + values.join(", ") + ")";
        this.executeNonQuery(query);
    } catch (e) {
        this.log("Error inserting data: " + e.message + " Data: " + JSON.stringify(data), 1);
    }
};

Database.prototype.update = function(table, data, where) {
    try {
        var set = [];
        for (var key in data) {
            set.push(key + "='" + data[key] + "'");
        }
        var query = "UPDATE " + table + " SET " + set.join(", ") + " WHERE " + where;
        this.executeNonQuery(query);
    } catch (e) {
        this.log("Error updating data: " + e.message + " Data: " + JSON.stringify(data) + " Where: " + where, 1);
    }
};

Database.prototype.delete = function(table, where) {
    try {
        var query = "DELETE FROM " + table + " WHERE " + where;
        this.executeNonQuery(query);
    } catch (e) {
        this.log("Error deleting data: " + e.message + " Where: " + where, 1);
    }
};

Database.prototype.batchInsert = function(table, dataArray) {
    for (var i = 0; i < dataArray.length; i++) {
        this.insert(table, dataArray[i]);
    }
};

Database.prototype.batchUpdate = function(table, dataArray, whereArray) {
    for (var i = 0; i < dataArray.length; i++) {
        this.update(table, dataArray[i], whereArray[i]);
    }
};

Database.prototype.batchDelete = function(table, whereArray) {
    for (var i = 0; i < whereArray.length; i++) {
        this.delete(table, whereArray[i]);
    }
};




要使用上述的 Database 類別來執行資料庫操作，你可以按照以下步驟進行。以下是一些具體的使用範例，包括創建 Database 對象，進行新增、修改、刪除、查詢操作，以及批次操作。

首先，確保你已經設置了正確的 ConnectionString。

## 1. 創建 Database 物件

In [ ]:
var db = new Database();

## 2. 執行查詢操作
假設你有一個名為 Users 的表格，並希望查詢所有記錄：

In [ ]:
var results = db.executeQuery("SELECT * FROM Users");
for (var i = 0; i < results.length; i++) {
    Watch.Log("User: " + results[i].Name, 3);  // 假設 Users 表格有一個名為 Name 的欄位
}


## 3. 新增操作
向 Users 表格插入一條新記錄：

In [ ]:
var newUser = {
    Name: "John Doe",
    Age: 30,
    Email: "john.doe@example.com"
};
db.insert("Users", newUser);


## 4. 修改操作
更新 Users 表格中某個用戶的年齡：

In [ ]:
var updateUser = {
    Age: 31
};
var whereClause = "Name='John Doe'";
db.update("Users", updateUser, whereClause);


## 5. 刪除操作
刪除 Users 表格中某個用戶：

In [ ]:
var whereClause = "Name='John Doe'";
db.delete("Users", whereClause);


## 6. 批次新增操作
批次插入多條記錄到 Users 表格：

In [ ]:
var usersArray = [
    { Name: "Alice Smith", Age: 25, Email: "alice.smith@example.com" },
    { Name: "Bob Johnson", Age: 28, Email: "bob.johnson@example.com" }
];
db.batchInsert("Users", usersArray);


## 7. 批次修改操作
批次更新 Users 表格中的記錄：

In [ ]:
var usersUpdateArray = [
    { Age: 26 },
    { Age: 29 }
];
var whereArray = [
    "Name='Alice Smith'",
    "Name='Bob Johnson'"
];
db.batchUpdate("Users", usersUpdateArray, whereArray);


## 8. 批次刪除操作
批次刪除 Users 表格中的記錄：

In [ ]:
var whereArray = [
    "Name='Alice Smith'",
    "Name='Bob Johnson'"
];
db.batchDelete("Users", whereArray);


## 9. 關閉資料庫連接
在操作完成後，請確保關閉資料庫連接：

In [ ]:
db.close();


為了更好地進行除錯，我們可以在每個函數中添加 try-catch 塊來捕獲和處理異常，並使用 Watch.Log 方法來記錄錯誤信息。這樣可以幫助我們更快地找到和解決問題。

以下是加入除錯功能的 Database 類別：

In [ ]:
var db = new Database();

try {
    // 查詢操作
    var results = db.executeQuery("SELECT * FROM Users");
    for (var i = 0; i < results.length; i++) {
        Watch.Log("User: " + results[i].Name, 3);  // 假設 Users 表格有一個名為 Name 的欄位
    }

    // 新增操作
    var newUser = {
        Name: "John Doe",
        Age: 30,
        Email: "john.doe@example.com"
    };
    db.insert("Users", newUser);

    // 修改操作
    var updateUser = {
        Age: 31
    };
    var whereClause = "Name='John Doe'";
    db.update("Users", updateUser, whereClause);

    // 刪除操作
    var whereClause = "Name='John Doe'";
    db.delete("Users", whereClause);

    // 批次新增操作
    var usersArray = [
        { Name: "Alice Smith", Age: 25, Email: "alice.smith@example.com" },
        { Name: "Bob Johnson", Age: 28, Email: "bob.johnson@example.com" }
    ];
    db.batchInsert("Users", usersArray);

    // 批次修改操作
    var usersUpdateArray = [
        { Age: 26 },
        { Age: 29 }
    ];
    var whereArray = [
        "Name='Alice Smith'",
        "Name='Bob Johnson'"
    ];
    db.batchUpdate("Users", usersUpdateArray, whereArray);

    // 批次刪除操作
    var whereArray = [
        "Name='Alice Smith'",
        "Name='Bob Johnson'"
    ];
    db.batchDelete("Users", whereArray);

} catch (e) {
    Watch.Log("An unexpected error occurred: " + e.message, 1);
} finally {
    db.close();
}


## insertAndGetId

In [ ]:

Database.prototype.insertAndGetId = function(insertQuery) {
    try {
        // 插入數據
        this.executeNonQuery(insertQuery);

        // 獲取新插入數據的ID
        var idQuery = "SELECT SCOPE_IDENTITY() AS NewId";
        var result = this.executeQuery(idQuery);
        if (result.length > 0) {
            return result[0].NewId;
        } else {
            this.log("無法獲取新插入數據的ID。", 1);
            return null;
        }
    } catch (e) {
        this.log("插入數據並獲取ID時出錯：" + e.message, 1);
        return null;
    }
};


In [ ]:

Database.prototype.updateAndGetId = function(table, whereClaus, identifier) {
    try {
        // 執行更新操作
        this.executeNonQuery(updateQuery);

        // 獲取剛剛更新的資料的ID，假設identifier是唯一標識符
        var idQuery = "SELECT id FROM "+table+" WHERE "+whereClaus+";";
        var result = this.executeQuery(idQuery);
        if (result.length > 0) {
            return result[0].id;
        } else {
            this.log("無法獲取更新後的資料ID。", 1);
            return null;
        }
    } catch (e) {
        this.log("更新資料並獲取ID時出錯：" + e.message, 1);
        return null;
    }
};

In [ ]:
// 使用示例
var db = new Database();

try {
    db.openConnection();

    var updateQuery = "UPDATE Users SET Age = 31 WHERE Name = 'John Doe'";
    var updatedId = db.updateAndGetId(updateQuery, 'John Doe');
    if (updatedId !== null) {
        Watch.Log("更新後資料的ID：" + updatedId, 3);
    }

} catch (e) {
    Watch.Log("發生意外錯誤：" + e.message, 1);
} finally {
    db.closeConnection();
}

In [ ]:

Database.prototype.executeQuery = function(query, params) {
    try {
        var cmd = new ActiveXObject("ADODB.Command");
        cmd.ActiveConnection = this.connection;
        cmd.CommandText = query;
        cmd.CommandType = 4; // adCmdStoredProc
        if (params) {
            for (var i = 0; i < params.length; i++) {
                cmd.Parameters.Append(cmd.CreateParameter(params[i].name, params[i].type, 1, params[i].size, params[i].value));
            }
        }
        var rs = cmd.Execute();
        var results = [];
        while (!rs.EOF) {
            var result = {};
            for (var i = 0; i < rs.Fields.Count; i++) {
                result[rs.Fields(i).Name] = rs.Fields(i).Value;
            }
            results.push(result);
            rs.MoveNext();
        }
        rs.Close();
        this.log("已執行查詢：" + query, 3);
        return results;
    } catch (e) {
        this.log("執行查詢時出錯：" + e.message + " 查詢：" + query, 1);
        return [];
    }
};

Database.prototype.executeNonQuery = function(query, params) {
    try {
        var cmd = new ActiveXObject("ADODB.Command");
        cmd.ActiveConnection = this.connection;
        cmd.CommandText = query;
        cmd.CommandType = 4; // adCmdStoredProc
        if (params) {
            for (var i = 0; i < params.length; i++) {
                cmd.Parameters.Append(cmd.CreateParameter(params[i].name, params[i].type, 1, params[i].size, params[i].value));
            }
        }
        cmd.Execute();
        this.log("已執行非查詢語句：" + query, 3);
    } catch (e) {
        this.log("執行非查詢語句時出錯：" + e.message + " 語句：" + query, 1);
    }
};